# Reset the Knowledge Graph

## Imports

### Import python packages

To start we'll load some useful python packages,
including some great stuff from langchain.


In [1]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

## Set up Neo4j and Langchain

### Gglobal variables

You will set up some global variable from the environment and some constants that
to use later.

In [2]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

In [3]:
print(f"Connecting to Neo4j at {NEO4J_URI} as {NEO4J_USERNAME}")

Connecting to Neo4j at neo4j://localhost:7687 as neo4j


### Prepare a knowledge graph interface

You can use the Langchain `Neo4jGraph` interface to send queries
to the Knowledge Graph.

In [4]:
# Create a knowledge graph using Langchain's Neo4j integration.
# This will be used for direct querying of the knowledge graph. 
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

# Clean up the graph to remove any existing data and indexes.

In [5]:
for constraint in kg.query('SHOW CONSTRAINTS'):
    kg.query(f"DROP CONSTRAINT {constraint['name']}")

In [6]:
for index in kg.query('SHOW INDEXES'):
    print(f"Removing index {index['name']}:")
    kg.query(f"""
        DROP INDEX `{index['name']}`
    """)

Removing index form_10k_chunks:


In [7]:
kg.query("""
        MATCH (all)
        DETACH DELETE all
""")

[]

## Basic Cypher Queries

### Script - helpful way to show schema

In [8]:
kg.refresh_schema()
print(kg.schema)

Node properties are the following:

Relationship properties are the following:

The relationships are the following:

